transform data in star model

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType

load fact_payment

In [0]:
payments = spark.table("default.staging_payments")

In [0]:
# write data to fact_payments
spark.sql("drop table if exists default.fact_payments")
payments.dropDuplicates(["PaymentId"]).write.format("delta").mode("overwrite").saveAsTable("default.fact_payments")

In [0]:
facts_payments = spark.table("default.fact_payments")

verify fact_payments

In [0]:
display(facts_payments.limit(10))

PaymentId,PaymentDate,Amount,RiderId
32,2021-12-01,9.0,1000
60,2020-12-01,6.47,1003
90,2021-01-01,9.0,1004
371,2022-02-01,9.0,1014
622,2021-01-01,14.35,1025
1014,2019-10-01,9.0,1040
1115,2022-01-01,9.0,1041
1931,2017-01-01,8.86,1072
1937,2017-07-01,19.49,1072
2098,2020-07-01,9.0,1081


load dim_rider

In [0]:
riders = spark.table("default.staging_riders")

In [0]:
# write data to dim_rider
spark.sql("drop table if exists default.dim_rider")
riders.dropDuplicates(["RiderId"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_rider")

verify dim_rider

In [0]:
dim_riders = spark.table("default.dim_rider")
display(dim_riders.limit(10))

RiderId,FirstName,LastName,Address,Birthday,AccountStartDate,AccountEndDate,IsMember
1236,Julia,Mack,64964 Michelle Wall Suite 298,1990-05-20,2020-04-24,null,true
1661,David,Jones,37516 Crawford Heights Suite 735,1996-10-16,2018-11-12,null,false
1810,John,Harvey,6701 Watkins Passage,1977-02-27,2019-02-15,null,true
1914,Shannon,Preston,88501 Zhang Valley Suite 076,1990-08-13,2018-08-25,null,true
1998,Veronica,Long,91058 Carson Circle,1990-03-12,2021-05-28,2022-01-01,true
2053,Ashley,Fields,14282 William Rest,1994-08-31,2019-04-29,null,true
2083,Craig,Bass,9517 Monica Springs Apt. 253,1992-01-15,2018-08-11,null,true
2097,Tina,Williams,33025 Vicki Parkway Apt. 245,2003-01-11,2020-11-04,null,true
2147,Casey,Brown,268 Horton Lakes,1993-04-01,2016-10-15,null,false
2277,Toni,Diaz,0758 Daniel Lakes Apt. 068,2001-10-24,2021-09-19,null,true


load dim_station

In [0]:
stations = spark.table("default.staging_stations")

In [0]:
# write data to dim_station
spark.sql("drop table if exists default.dim_station")
stations.dropDuplicates(["StationId"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_station")

verify dim_station

In [0]:
dim_stations = spark.table("default.dim_station")
display(dim_stations.limit(10))

StationId,StationName,Latitude,Longitude
13192,Halsted St & Dickens Ave,41.919936,-87.64883
KA1504000162,Clark St & Lunt Ave,42.009074,-87.67419
15634,Western Ave & Roscoe St,41.943093,-87.6873335
15539,Desplaines St & Jackson Blvd,41.878161166666665,-87.64428766666668
TA1305000022,Orleans St & Merchandise Mart Plaza,41.888243,-87.63639
KA1503000075,DuSable Museum,41.79156801058,-87.60785217739999
13133,Damen Ave & Cortland St,41.915983000000004,-87.677335
TA1305000009,Clark St & Ida B Wells Dr,41.8759326655,-87.63058453549999
KA1504000135,Wells St & Elm St,41.90320733333333,-87.63461616666666
17660,California Ave & Cortez St,41.900363,-87.696704


load fact_trip

In [0]:
trips = spark.table("default.staging_trips")

In [0]:
fact_trips = trips.alias("tbl1").join(riders.alias("tbl2"), col("tbl1.RiderId") == col("tbl2.RiderId"), "left") \
            .withColumn('Duration', round((unix_timestamp("EndedAt") - unix_timestamp('StartAt'))/60)) \
            .withColumn('RiderAge', round((unix_timestamp("AccountStartDate") - unix_timestamp('Birthday'))/3600/24)) \
            .withColumnRenamed('StartAt', 'StartTimeId') \
            .withColumnRenamed('EndedAt', 'EndTimeId') \
            .select("TripId", col("tbl1.RiderId"), "RideableType", "StartStationId", "EndStationId", "StartTimeId", "EndTimeId", "Duration", "RiderAge")
            
display(fact_trips.limit(10))

TripId,RiderId,RideableType,StartStationId,EndStationId,StartTimeId,EndTimeId,Duration,RiderAge
89E7AA6C29227EFF,71934,classic_bike,525,660,2021-02-12T16:14:56Z,2021-02-12T16:21:43Z,7.0,13638.0
0FEFDE2603568365,47854,classic_bike,525,16806,2021-02-14T17:52:38Z,2021-02-14T18:12:09Z,20.0,13523.0
E6159D746B2DBB91,70870,electric_bike,KA1503000012,TA1305000029,2021-02-09T19:10:18Z,2021-02-09T19:19:10Z,9.0,12051.0
B32D3199F1C2E75B,58974,classic_bike,637,TA1305000034,2021-02-02T17:49:41Z,2021-02-02T17:54:06Z,4.0,7107.0
83E463F23575F4BF,39608,electric_bike,13216,TA1309000055,2021-02-23T15:07:23Z,2021-02-23T15:22:37Z,15.0,25961.0
BDAA7E3494E8D545,36267,electric_bike,18003,KP1705001026,2021-02-24T15:43:33Z,2021-02-24T15:49:05Z,6.0,10176.0
A772742351171257,50104,classic_bike,KP1705001026,KP1705001026,2021-02-01T17:47:42Z,2021-02-01T17:48:33Z,1.0,10697.0
295476889D9B79F8,19618,classic_bike,18003,18003,2021-02-11T18:33:53Z,2021-02-11T18:35:09Z,1.0,7767.0
362087194BA4CC9A,16732,classic_bike,KP1705001026,KP1705001026,2021-02-27T15:13:39Z,2021-02-27T15:36:36Z,23.0,5626.0
21630F715038CCB0,57068,classic_bike,KP1705001026,KP1705001026,2021-02-20T08:59:42Z,2021-02-20T09:17:04Z,17.0,16461.0


In [0]:
# write data to fact_trip
spark.sql("drop table if exists default.fact_trip")
fact_trips.write.format("delta").mode("overwrite").saveAsTable("default.fact_trip")

load dim_time

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
# Get min date from trips
min_date = trips.selectExpr('MIN(StartAt) AS StartedAt').first().asDict()['StartedAt']
# Add date range as max start_at from trip plus 5 years
max_date = trips.selectExpr('DATEADD(year, 5, MAX(StartAt)) AS StartedAt').first().asDict()['StartedAt']
expression = f"sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)"
dim_time = spark.createDataFrame([(1,)], ["TimeId"])

dim_time = dim_time.withColumn("DateInit", f.explode(f.expr(expression)))
dim_time = dim_time.withColumn("Date", f.to_timestamp(dim_time.DateInit, "yyyy-MM-dd"))

dim_time = dim_time \
            .withColumn("DayOfWeek", f.dayofweek(dim_time.Date)) \
            .withColumn("DayOfMonth", f.dayofmonth(dim_time.Date)) \
            .withColumn("WeekOfYear", f.weekofyear(dim_time.Date)) \
            .withColumn("Year", f.year(dim_time.Date)) \
            .withColumn("Quarter", f.quarter(dim_time.Date)) \
            .withColumn("Month", f.month(dim_time.Date)) \
            .withColumn("TimeId", dim_time.Date.cast(StringType())) \
            .drop(f.col("DateInit"))

verify dim_time

In [0]:
display(dim_time.limit(10))

TimeId,Date,DayOfWeek,DayOfMonth,WeekOfYear,Year,Quarter,Month
2021-02-01 00:00:00,2021-02-01T00:00:00Z,2,1,5,2021,1,2
2021-02-02 00:00:00,2021-02-02T00:00:00Z,3,2,5,2021,1,2
2021-02-03 00:00:00,2021-02-03T00:00:00Z,4,3,5,2021,1,2
2021-02-04 00:00:00,2021-02-04T00:00:00Z,5,4,5,2021,1,2
2021-02-05 00:00:00,2021-02-05T00:00:00Z,6,5,5,2021,1,2
2021-02-06 00:00:00,2021-02-06T00:00:00Z,7,6,5,2021,1,2
2021-02-07 00:00:00,2021-02-07T00:00:00Z,1,7,5,2021,1,2
2021-02-08 00:00:00,2021-02-08T00:00:00Z,2,8,6,2021,1,2
2021-02-09 00:00:00,2021-02-09T00:00:00Z,3,9,6,2021,1,2
2021-02-10 00:00:00,2021-02-10T00:00:00Z,4,10,6,2021,1,2


In [0]:
# write data to dim_time
spark.sql("drop table if exists default.dim_time")
dim_time.write.format("delta").mode("overwrite").saveAsTable("default.dim_time")